In [4]:
import os
from pathlib import Path

def find_project_root(start=None, markers=("原始数据", "新数据", "README.md", ".git", "requirements.txt")):
    cur = Path(start or os.getcwd()).resolve()
    for p in [cur] + list(cur.parents):
        if any((p / m).exists() for m in markers):
            return p
    raise RuntimeError(f"Cannot find project root from {cur}. Please check markers or working dir.")

BASE_DIR_0 = find_project_root()

In [5]:
# pip install pandas numpy matplotlib scipy statsmodels openpyxl
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager, rcParams

# ===== 中文字体设置（Windows 推荐）=====
rcParams["font.sans-serif"] = ["Microsoft YaHei"]   # 微软雅黑
rcParams["axes.unicode_minus"] = False               # 解决负号显示为方块的问题

# ---------- 配置 ----------
BASE_DIR = BASE_DIR_0/"整合数据"

# 六行业文件（你只要改最后中文）
files = {
    "餐饮": BASE_DIR / "餐饮_panel_processed_sim.csv",
    "动漫": BASE_DIR / "动漫_panel_processed_sim.csv",
    "家电": BASE_DIR / "家电_panel_processed_sim.csv",
    "旅游": BASE_DIR / "旅游_panel_processed_sim.csv",
    "汽车": BASE_DIR / "汽车_panel_processed_sim.csv",
    "文娱": BASE_DIR / "文娱_panel_processed_sim.csv",
}

OUT_DIR = BASE_DIR / "_EDA_outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

DATE_COL = "date"

CORE_VARS = ["Y", "D_star", "OI", "ET", "premium", "mom_1w"]
# 如果你的列名里 D 不叫这个，改这里

# ---------- 工具函数 ----------
def read_csv_any(path: Path) -> pd.DataFrame:
    for enc in ["utf-8-sig", "utf-8", "gbk"]:
        try:
            return pd.read_csv(path, sep=None, engine="python", encoding=enc)
        except Exception:
            continue
    raise RuntimeError(f"Cannot read: {path}")

def to_datetime_safe(s: pd.Series) -> pd.Series:
    # 你的 date 像 "2025/5/15"；to_datetime能处理
    return pd.to_datetime(s, errors="coerce")

def basic_health_report(df: pd.DataFrame) -> pd.DataFrame:
    na_count = df.isna().sum()
    na_ratio = df.isna().mean()
    dtypes = df.dtypes.astype(str)
    out = pd.concat([na_count.rename("na_count"),
                     na_ratio.rename("na_ratio"),
                     dtypes.rename("dtype")], axis=1)
    return out.sort_values("na_ratio", ascending=False)

def summary_stats(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    # 分位数 + 均值方差 + 偏度峰度
    use = df[cols].apply(pd.to_numeric, errors="coerce")
    desc = use.describe(percentiles=[0.01,0.05,0.25,0.5,0.75,0.95,0.99]).T
    desc["skew"] = use.skew(numeric_only=True)
    desc["kurt"] = use.kurt(numeric_only=True)
    return desc

def save_hist_box(df: pd.DataFrame, cols: list, out_dir: Path, prefix: str):
    use = df[cols].apply(pd.to_numeric, errors="coerce")
    for c in cols:
        x = use[c].dropna().values
        if len(x) == 0:
            continue

        # hist
        plt.figure(figsize=(7,4))
        plt.hist(x, bins=50)
        plt.title(f"{prefix} - {c} histogram")
        plt.xlabel(c); plt.ylabel("count")
        plt.tight_layout()
        plt.savefig(out_dir / f"{prefix}_{c}_hist.png", dpi=150)
        plt.close()

        # box
        plt.figure(figsize=(5,3))
        plt.boxplot(x, vert=False)
        plt.title(f"{prefix} - {c} boxplot")
        plt.xlabel(c)
        plt.tight_layout()
        plt.savefig(out_dir / f"{prefix}_{c}_box.png", dpi=150)
        plt.close()

def corr_tables(df: pd.DataFrame, cols: list) -> tuple[pd.DataFrame, pd.DataFrame]:
    use = df[cols].apply(pd.to_numeric, errors="coerce")
    pearson = use.corr(method="pearson")
    spearman = use.corr(method="spearman")
    return pearson, spearman

def lead_lag_corr(df: pd.DataFrame, y: str, x: str, max_lag: int = 20) -> pd.DataFrame:
    """
    corr(y_t, x_{t-k}), k=0..max_lag （x滞后）
    """
    tmp = df[[DATE_COL, y, x]].copy()
    tmp[y] = pd.to_numeric(tmp[y], errors="coerce")
    tmp[x] = pd.to_numeric(tmp[x], errors="coerce")
    tmp = tmp.dropna(subset=[y]).sort_values(DATE_COL)

    rows = []
    for k in range(max_lag + 1):
        corr = tmp[y].corr(tmp[x].shift(k))
        rows.append({"lag": k, "corr": corr})
    return pd.DataFrame(rows)

def plot_timeseries(df: pd.DataFrame, cols: list, out_path: Path, title: str):
    tmp = df[[DATE_COL] + cols].copy()
    tmp = tmp.sort_values(DATE_COL)
    plt.figure(figsize=(10,4))
    for c in cols:
        plt.plot(tmp[DATE_COL], pd.to_numeric(tmp[c], errors="coerce"), label=c)
    plt.title(title)
    plt.xlabel("date")
    plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    plt.close()

def rolling_stats(df: pd.DataFrame, col: str, window: int = 30) -> pd.DataFrame:
    tmp = df[[DATE_COL, col]].copy().sort_values(DATE_COL)
    x = pd.to_numeric(tmp[col], errors="coerce")
    out = pd.DataFrame({
        "date": tmp[DATE_COL],
        f"{col}_roll_mean_{window}": x.rolling(window, min_periods=5).mean(),
        f"{col}_roll_std_{window}": x.rolling(window, min_periods=5).std(ddof=1),
    })
    return out

def largest_jumps(df: pd.DataFrame, col: str, topk: int = 10) -> pd.DataFrame:
    tmp = df[[DATE_COL, col]].copy().sort_values(DATE_COL)
    x = pd.to_numeric(tmp[col], errors="coerce")
    tmp["diff"] = x.diff()
    tmp["abs_diff"] = tmp["diff"].abs()
    return tmp.sort_values("abs_diff", ascending=False).head(topk)

# ---------- 主循环：每行业输出一套 EDA ----------
all_summary = []
all_health = []

for ind, path in files.items():
    if not path.exists():
        print(f"[WARN] missing file: {path}")
        continue

    df = read_csv_any(path)
    df.columns = [str(c).strip() for c in df.columns]

    # date parse
    if DATE_COL not in df.columns:
        # 兜底：第一列当 date
        df = df.rename(columns={df.columns[0]: DATE_COL})
    df[DATE_COL] = to_datetime_safe(df[DATE_COL])
    df = df.dropna(subset=[DATE_COL]).sort_values(DATE_COL).reset_index(drop=True)

    ind_out = OUT_DIR / ind
    ind_out.mkdir(parents=True, exist_ok=True)

    # 1) health
    health = basic_health_report(df)
    health.to_csv(ind_out / "missing_report.csv", encoding="utf-8-sig")
    all_health.append(health.assign(industry=ind).reset_index().rename(columns={"index":"var"}))

    # 日期覆盖
    with open(ind_out / "date_coverage.txt", "w", encoding="utf-8") as f:
        f.write(f"rows={len(df)}\n")
        f.write(f"start={df[DATE_COL].min()}\n")
        f.write(f"end={df[DATE_COL].max()}\n")
        dup = df[DATE_COL].duplicated().sum()
        f.write(f"duplicated_dates={dup}\n")

    # 2) summary stats（只对数值列做；这里重点输出核心变量 + 所有 *_star）
    star_cols = [c for c in df.columns if c.endswith("_star")]
    cols_for_stats = [c for c in CORE_VARS if c in df.columns] + star_cols
    cols_for_stats = list(dict.fromkeys(cols_for_stats))  # 去重保序

    stats = summary_stats(df, cols_for_stats)
    stats.to_csv(ind_out / "summary_stats.csv", encoding="utf-8-sig")
    stats2 = stats.copy()
    stats2["industry"] = ind
    all_summary.append(stats2.reset_index().rename(columns={"index":"var"}))

    # 3) distribution plots（核心变量）
    save_hist_box(df, [c for c in CORE_VARS if c in df.columns], ind_out, prefix=ind)

    # 4) correlations（核心变量 + controls 可按需扩展）
    corr_cols = [c for c in CORE_VARS if c in df.columns]
    pearson, spearman = corr_tables(df, corr_cols)
    pearson.to_csv(ind_out / "corr_pearson.csv", encoding="utf-8-sig")
    spearman.to_csv(ind_out / "corr_spearman.csv", encoding="utf-8-sig")

    # 5) lead-lag correlations：Y vs D/OI/ET
    if "Y" in df.columns:
        for x in ["D_total_norm", "OI", "ET"]:
            if x in df.columns:
                ll = lead_lag_corr(df, "Y", x, max_lag=20)
                ll.to_csv(ind_out / f"leadlag_Y_vs_{x}.csv", index=False, encoding="utf-8-sig")

                plt.figure(figsize=(7,4))
                plt.plot(ll["lag"], ll["corr"], marker="o")
                plt.title(f"{ind} lead-lag corr: Y_t vs {x}_{{t-lag}}")
                plt.xlabel("lag"); plt.ylabel("corr")
                plt.tight_layout()
                plt.savefig(ind_out / f"leadlag_Y_vs_{x}.png", dpi=150)
                plt.close()

    # 6) core timeseries plot（用标准化变量同轴画）
    plot_cols = [c for c in ["Y", "D_total_norm", "OI", "ET"] if c in df.columns]
    if plot_cols:
        plot_timeseries(df, plot_cols, ind_out / "timeseries_core.png", f"{ind} core series")

    # 7) rolling stats & largest jumps（对 Y）
    if "Y" in df.columns:
        rs = rolling_stats(df, "Y", window=30)
        rs.to_csv(ind_out / "rolling_Y_30.csv", index=False, encoding="utf-8-sig")

        lj = largest_jumps(df, "Y", topk=15)
        lj.to_csv(ind_out / "largest_jumps_Y.csv", index=False, encoding="utf-8-sig")

print("EDA outputs saved to:", OUT_DIR)

# 汇总输出（便于你横向比较六行业）
if all_summary:
    pd.concat(all_summary, ignore_index=True).to_csv(OUT_DIR / "_ALL_summary_stats.csv", index=False, encoding="utf-8-sig")
if all_health:
    pd.concat(all_health, ignore_index=True).to_csv(OUT_DIR / "_ALL_missing_reports.csv", index=False, encoding="utf-8-sig")


EDA outputs saved to: C:\Users\17514\Desktop\研一\统计基础\作业\大作业\整合数据\_EDA_outputs


热力图

In [6]:
# pip install pandas numpy matplotlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib import rcParams

# ===== 中文字体设置（Windows 推荐）=====
rcParams["font.sans-serif"] = ["Microsoft YaHei"]   # 微软雅黑
rcParams["axes.unicode_minus"] = False              # 解决负号显示为方块的问题

# ===================== 配置区 =====================
OUT_DIR  = BASE_DIR / "_EDA_outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 你的六个行业（需与 _EDA_outputs/ 子目录一致）
INDUSTRIES = ["餐饮", "动漫", "家电", "旅游", "汽车", "文娱"]

# 读哪个相关矩阵文件：corr_pearson.csv 或 corr_spearman.csv
CORR_FILE = "corr_pearson.csv"

# 统一变量顺序（保证跨行业可比）
VAR_ORDER = ["Y", "D_star", "OI", "ET", "premium", "mom_1w"]

# 是否在格子里显示数值（想更干净就改 False）
ANNOTATE = True

# ===================== 工具函数 =====================
def read_corr(path: Path) -> pd.DataFrame:
    """稳健读取相关矩阵csv：首列为index"""
    for enc in ["utf-8-sig", "utf-8", "gbk"]:
        try:
            df = pd.read_csv(path, index_col=0, encoding=enc)
            df.index = df.index.astype(str).str.strip()
            df.columns = df.columns.astype(str).str.strip()
            return df
        except Exception:
            continue
    raise RuntimeError(f"无法读取文件：{path}")

def align_corr(df: pd.DataFrame, var_order: list[str]) -> pd.DataFrame:
    """按给定变量顺序对齐（只取交集）"""
    keep = [v for v in var_order if v in df.index and v in df.columns]
    return df.loc[keep, keep]

# ===================== 主流程：读取六行业矩阵 =====================
corr_mats = {}
missing = []

for ind in INDUSTRIES:
    f = OUT_DIR / ind / CORR_FILE
    if not f.exists():
        missing.append(str(f))
        continue
    df = read_corr(f)
    df = align_corr(df, VAR_ORDER)
    corr_mats[ind] = df

if not corr_mats:
    raise RuntimeError("没有读到任何行业的相关矩阵文件，请检查路径与文件名。")

if missing:
    print("[WARN] 以下行业文件缺失，将在图中留空：")
    for m in missing:
        print(" ", m)

# ===================== 作图：2×3 总热力图 =====================
fig, axes = plt.subplots(2, 3, figsize=(16, 9), constrained_layout=True)
axes = axes.flatten()

vmin, vmax = -1.0, 1.0  # 固定色阶，便于跨行业比较
im_for_cbar = None

for ax, ind in zip(axes, INDUSTRIES):
    if ind not in corr_mats:
        ax.axis("off")
        ax.set_title(f"{ind}（缺失）")
        continue

    df = corr_mats[ind]
    mat = df.values

    im = ax.imshow(mat, vmin=vmin, vmax=vmax, aspect="equal")
    im_for_cbar = im

    ax.set_title(ind, fontsize=12)
    ax.set_xticks(range(len(df.columns)))
    ax.set_xticklabels(df.columns, rotation=45, ha="right")
    ax.set_yticks(range(len(df.index)))
    ax.set_yticklabels(df.index)

    # 格子边框（更清晰）
    ax.set_xticks(np.arange(-.5, len(df.columns), 1), minor=True)
    ax.set_yticks(np.arange(-.5, len(df.index), 1), minor=True)
    ax.grid(which="minor", linestyle="-", linewidth=0.5)
    ax.tick_params(which="minor", bottom=False, left=False)

    # 数值标注
    if ANNOTATE:
        for i in range(mat.shape[0]):
            for j in range(mat.shape[1]):
                ax.text(j, i, f"{mat[i, j]:.2f}", ha="center", va="center", fontsize=8)

# 如果子图多出来（理论上不会），关掉
for k in range(len(INDUSTRIES), len(axes)):
    axes[k].axis("off")

# 统一色条
if im_for_cbar is not None:
    cbar = fig.colorbar(im_for_cbar, ax=axes, fraction=0.02, pad=0.01)
    cbar.set_label("Pearson 相关系数")

out_path = OUT_DIR / "corr_heatmap_6industries_pearson.png"
plt.savefig(out_path, dpi=200)
plt.close()

print("已保存：", out_path)


已保存： C:\Users\17514\Desktop\研一\统计基础\作业\大作业\整合数据\_EDA_outputs\corr_heatmap_6industries_pearson.png
